In [ ]:
import sys
sys.path.insert(0, "D:/dip/Adaptation-of-Action-Space-for-Reinforcement-Learning/")
import os
import re
import numpy as np
import matplotlib.pyplot as plt

import src.util.data as data_lib

CURRENT_DIR = os.getcwd()
DATA_TEMPLATE = "data_.*\.json.zip"

## user must set BATCH_SIZE and implement convert_episode to improve performance
BATCH_SIZE = 200
def convert_episode(episode):
    '''
    episode template
    remove any useless information to deplete the file
    {
        "id":0,
        "states":[],
        "actions":[],
        "actors_actions":[],
        "ndn_actions":[],
        "rewards":[],
        "action_space_sizes":[]
    }
    '''
    del episode["states"]
    del episode["actors_actions"]
    del episode["ndn_actions"]
    del episode["action_space_sizes"]
    episode["rewards"] = np.sum(episode["rewards"])

    
## -----------------------------------------------------------------------------
    
def get_data_filenames(deplete=True):
    all_filenames = os.listdir()
        
    result_filenames = []
    
    for filename in all_filenames:
        match = re.search(DATA_TEMPLATE, filename)
        if match is not None:
            depleted_file = "{}{}".format("light_",filename[:len(filename)-4])
            if not os.path.exists(depleted_file) and deplete:
                data_lib.deplete(filename, convert_episode)
                result_filenames.append("{}/{}".format(CURRENT_DIR, depleted_file))
            elif os.path.exists(depleted_file):
                result_filenames.append("{}/{}".format(CURRENT_DIR, depleted_file))
            else:   
                result_filenames.append("{}/{}".format(CURRENT_DIR, filename))
    return result_filenames

def load_data(filename):
    
    data_dict = data_lib.load(filename).data
    
    return data_dict


def get_episode_data(data_dict, field):
    episodes = data_dict['simulation']['episodes']
    data = list(data[field] for data in episodes)
    return np.array(data)

def apply_func_to_window(data, window_size, func):
    data_lenght = len(data)
    window_size = min(window_size, data_lenght)
    if window_size == 0:
        window_size = (data_lenght * .1)
    res = []
    for i in range(data_lenght):
        start = int(max(i - window_size / 2, 0))
        end = int(min(i + window_size / 2, data_lenght - 1))
        if start == end:
            continue
        res.append(func(data[start:end]))

    return np.array(res)

